In [34]:
import sys
sys.path.append('./')

import analytics as an
from analytics.experiment_modelling.cyclic_voltammetry import CyclicVoltammogram

import pandas as pd
import plotly.express as px
import numpy as np
import scipy as sc

In [2]:

my_cv = (CyclicVoltammogram
         .from_aftermath("../test_trajectories/cyclic_voltammetry/aftermath1.csv", scan_rate=10000)
         )


In [3]:
my_cv

In [43]:
data = (my_cv
        ._data)

my_cv.show_current_potential()

In [39]:
data = (my_cv
        ._data
        )

cross_indices = (data
                 .assign(sign_diff = lambda x: np.sign(x['current']).diff())
                 .query('sign_diff != 0 and sign_diff.notna()')
                 .index
                 )

window1 = [i for i in range(cross_indices[1]-5, cross_indices[1]+5)]

window_data = data.query('index in @window1')

px.line(window_data, x='time', y='current', title='Cyclic Voltammetry', markers=True).show()


In [38]:

spline = sc.interpolate.UnivariateSpline(window_data['time'], window_data['current'], k=3)
root = spline.roots()
root

array([0.76015558])

In [5]:
my_cv.show_current_potential(template = 'plotly_dark', height=600, width=1100)

In [6]:
my_cv.show_current_time(template='plotly_dark')

In [7]:
my_cv.get_charge_passed()

,direction,segment,cycle,anodic_charge,cathodic_charge
0,reduction,1,1,3.701305e-11,3.210752e-09
1,oxidation,2,1,3.105005e-09,5.835596e-11
2,reduction,3,2,5.084812e-11,3.164704e-09
3,oxidation,4,2,2.954910e-09,7.550363e-11
4,reduction,5,3,4.105527e-11,3.106489e-09
5,oxidation,6,3,2.885522e-09,7.118698e-11
6,reduction,7,4,3.621879e-11,2.999927e-09


In [8]:
my_cv.show_charge_passed(width=800, height=500, template='plotly_dark')

In [9]:
data = my_cv.drop_cycles(keep=[3]).show_current_time(template='plotly_dark', width=1200, height=900).data

data

,potential,current,time,direction,segment,cycle
1455,0.428379,3.038612e-09,0.726469,reduction,5,3
1456,0.426417,2.759369e-09,0.726968,reduction,5,3
1457,0.421430,2.550996e-09,0.727467,reduction,5,3
1458,0.416421,2.392636e-09,0.727966,reduction,5,3
1459,0.411433,2.260903e-09,0.728465,reduction,5,3
...,...,...,...,...,...,...
2132,0.408381,2.638567e-09,1.063758,oxidation,6,3
2133,0.413372,2.666598e-09,1.064257,oxidation,6,3
2134,0.418369,2.697193e-09,1.064756,oxidation,6,3
2135,0.423373,2.728502e-09,1.065255,oxidation,6,3
